In [3]:
import os
import sys

from pathbook.pathbook import *
labels = ['klikun', 'maliy', 'shipun']
labeldict = {'klikun':0, 'maliy':1, 'shipun':2}

import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil

### annotating

In [12]:
df = pd.read_csv(path_test_annotation)
for img in df['path'].unique():
    with open(path_test_dataset / img.replace('.jpg','.txt'),'w') as txt:
        info = df[df['path']==img].copy()
        info['x']=(info.x_min+info.x_max)/2
        info['y']=(info.y_min+info.y_max)/2
        info['w']=(-info.x_min+info.x_max)
        info['h']=(-info.y_min+info.y_max)
        for idx, row in info.iterrows():
            print(labeldict[row.label],row.x,row.y,row.w,row.h,file=txt)

In [6]:
for s in [path_val_dataset, path_train_dataset]:
    for dir in os.listdir(s):
        for path in tqdm(os.listdir(os.path.join(s,dir))):
            if path[-4:]=='.txt':
                path = os.path.join(s,dir,path)
                with open(path,'r') as file:
                    lines = file.readlines()
                with open(path,'w') as file:
                    for line in lines:
                        print(str(labeldict[path.split('/')[-1].split('-')[1]]),line[2:],file=file)

100%|██████████| 5100/5100 [00:01<00:00, 4074.66it/s]


In [21]:
for _ in [path_val_dataset, path_train_dataset]:
    for img in tqdm(os.listdir(_)):
        if img[-4:]=='.jpg':
            label = img.split('-')[1]
            cv2.imwrite(os.path.join(_,label,img), cv2.imread(os.path.join(_,img)))

100%|██████████| 15357/15357 [01:44<00:00, 147.12it/s]


### segmentations from initial dataset

In [5]:
columns=['set','path','class_name','class_id','x_min','y_min','x_max','y_max']
train_array = []
val_array = []

np.random.seed(17)
val_p = 0.15 

# iterate through the initial formated for classification dataset
dataset = path_initial_train_dataset
for label_idx, label in enumerate(labels):
    mask_dir = os.path.join(dataset, label, 'masks')
    image_dir = os.path.join(dataset, label, 'images')

    for file_idx, mask in enumerate(tqdm(os.listdir(mask_dir))):
        impath = os.path.join(image_dir, mask.replace('.png','.jpg'))
        if not os.path.exists(impath):
            # print(impath)
            continue # drop masks without images

        image = cv2.imread(impath)
        mask = cv2.imread(os.path.join(mask_dir, mask))

        h, w, _ = image.shape
        h_, w_, _ = mask.shape
        if not (h == h_ and w == w_):
            continue # drop images with uncorrest masks

        save_dir=''
        save_cls_dir=''
        val = (np.random.random() < val_p)
        if val:
            save_dir = path_val_dataset
            save_cls_dir = path_cls_val_dataset
        else:
            save_dir = path_train_dataset
            save_cls_dir = path_cls_train_dataset

        # name format: "[scource: initial|extra|augmented]-[label: klikun|maliy|shipun]-[ingroup index]]"
        name = f'initial-{label}-{file_idx}'
        path = os.path.join(save_dir, label, name)

        # saving image and annotation for segmentation task
        cv2.imwrite(path+'.jpg', image)
        with open(path+'.txt','w') as labelfile:

            imgray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            colors = np.unique(imgray)[1:]
            for color_idx, color in enumerate(colors):
                #choose one color (one object) from mask and find contours
                _, thresh = cv2.threshold(imgray, color, color, type=cv2.THRESH_TOZERO_INV)
                _, thresh = cv2.threshold(thresh, color-1, color, type=cv2.THRESH_TOZERO)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                superseg = []
                for contour in contours:
                    seg = (contour[:,0]/[w,h]).flatten().tolist()
                    superseg += seg
                    # if mode=='seg':
                    print(0,*seg,file=labelfile) # save each spot for segmentation task

                # contours coords
                x = superseg[0::2]
                x_min = min(x)
                x_max = max(x)

                y = superseg[1::2]
                y_min = min(y)
                y_max = max(y)

                w_ = x_max-x_min
                h_ = y_max-y_min

                # saving different crops for classification task
                cls_path = os.path.join(save_cls_dir, label, name+f"-{color_idx}")
                cls_a_path = os.path.join(save_cls_dir, label, f'augmented-{label}-{file_idx}-{color_idx}')
                cv2.imwrite(cls_a_path+'-1'+'.jpg', 
                            image[int(y_min*h) : int(y_max*h), 
                                  int(x_min*w) : int(x_max*w)])
                if w_ < 0.8 and h_ < 0.8:
                    cv2.imwrite(cls_path+'-1.2'+'.jpg', 
                            image[int(max((y_min-0.1*h_),0)*h) : int(min((y_max+0.1*h_),1)*h), 
                                  int(max((x_min-0.1*w_),0)*w) : int(min((x_max+0.1*w_),1)*w)])
                if w_ < 0.6 and h_ < 0.6:
                    cv2.imwrite(cls_a_path+'-1.4'+'.jpg', 
                            image[int(max((y_min-0.2*h_),0)*h) : int(min((y_max+0.2*h_),1)*h), 
                                  int(max((x_min-0.2*w_),0)*w) : int(min((x_max+0.2*w_),1)*w)])

                # if mode=='det':
                #     print(label_idx,min(x),min(y),max(x),max(y)) # save bboxes for detection

                if val:
                    val_array.append(['val',path+'.jpg',label,labeldict[label],x_min,y_min,x_max,y_max])
                else:
                    train_array.append(['train',path+'.jpg',label,labeldict[label],x_min,y_min,x_max,y_max])

# if mode!='cls': #save annotations for detection
pd.DataFrame(train_array, columns=columns).to_csv(path_train_annotation,index=False)
pd.DataFrame(val_array, columns=columns).to_csv(path_val_annotation,index=False)

100%|██████████| 3011/3011 [01:57<00:00, 25.70it/s]


### test repair

In [ ]:
df = pd.read_csv(path_test_annotation)

In [35]:
df.groupby('path').count().idxmax()

set           251.jpg
x_min         251.jpg
y_min         251.jpg
x_max         251.jpg
y_max         251.jpg
class_name    251.jpg
class_id      251.jpg
dtype: object

In [42]:
for img in df['path'].unique()[:]:
    path = os.path.join(path_test_dataset,img.replace('.jpg','.txt'))
    lines=[]
    with open(path,'r') as f:
        lines = f.readlines()
    with open(path,'w') as f:
        for line, id in zip(lines,df[df['path']==img]['class_id']):
            print(str(id) + line[1:-1],file=f)

### cls dataset
crops of initial images with 20% padings:
cls_image_size / bbox_size = 1.4

In [1]:
!rm -rf ../dataset/classification
!mkdir ../dataset/classification
!mkdir ../dataset/classification/test
!mkdir ../dataset/classification/test/klikun
!mkdir ../dataset/classification/test/maliy
!mkdir ../dataset/classification/test/shipun
!mkdir ../dataset/classification/val
!mkdir ../dataset/classification/val/klikun
!mkdir ../dataset/classification/val/maliy
!mkdir ../dataset/classification/val/shipun
!mkdir ../dataset/classification/train
!mkdir ../dataset/classification/train/klikun
!mkdir ../dataset/classification/train/maliy
!mkdir ../dataset/classification/train/shipun

In [6]:
df_train=pd.read_csv(path_train_annotation)
df_val=pd.read_csv(path_val_annotation)
df_test=pd.read_csv(path_test_annotation)

dataset = path_initial_train_dataset
for label_idx, label in enumerate(labels):
    mask_dir = os.path.join(dataset, label, 'masks')
    image_dir = os.path.join(dataset, label, 'images')

    for file_idx, mask in enumerate(tqdm(os.listdir(mask_dir))):
        impath = os.path.join(image_dir, mask.replace('.png','.jpg'))
        if not os.path.exists(impath):
            # print(impath)
            continue # drop masks without images

        image = cv2.imread(impath)
        mask = cv2.imread(os.path.join(mask_dir, mask))

        h, w, _ = image.shape
        h_, w_, _ = mask.shape
        if not (h == h_ and w == w_):
            continue # drop images with uncorrect masks

        # name format: "[scource: initial|extra|augmented]-[label: klikun|maliy|shipun]-[ingroup index]]"
        name = f'initial-{label}-{file_idx}'

        save_cls_dir=''
        if label+'/'+name+'.jpg' in df_train['path'].values:
            save_cls_dir = path_cls_train_dataset 
        elif label+'/'+name+'.jpg' in df_val['path'].values:
            save_cls_dir = path_cls_val_dataset
        elif label+'/'+name+'.jpg' in df_test['path'].values:
            save_cls_dir = path_cls_test_dataset
        else:
            # print(impath, name)
            continue

        imgray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        for color_idx, color in enumerate(np.unique(imgray)[1:]):
            _, thresh = cv2.threshold(imgray, color, color, type=cv2.THRESH_TOZERO_INV)
            _, thresh = cv2.threshold(thresh, color-1, color, type=cv2.THRESH_TOZERO)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # returns only most parental (external) contours
            
            superseg = []
            for contour in contours:
                seg = (contour[:,0]/[w,h]).flatten().tolist()
                superseg += seg

            # contours coords
            x = superseg[0::2]
            x_min = min(x)
            x_max = max(x)
            x_c = (x_min + x_max)/2*w

            y = superseg[1::2]
            y_min = min(y)
            y_max = max(y)
            y_c = (y_max+y_min)/2*h

            size = max((y_max-y_min)*h, (x_max-x_min)*w)

            X_min = int(x_c - 1.4*size/2)
            X_max = int(x_c + 1.4*size/2)
            Y_min = int(y_c - 1.4*size/2)
            Y_max = int(y_c + 1.4*size/2)

            if X_min < 0: 
                X_max = min(X_max - X_min, w)
                X_min = 0
            if X_max > w:
                X_min = max(X_min - X_max + w, 0)
                X_max = w
            if Y_min < 0:
                Y_max = min(Y_max - Y_min, h)
                Y_min = 0
            if Y_max > h:
                Y_min = max(Y_min - Y_max + h, 0)
                Y_max = h


            cv2.imwrite(os.path.join(save_cls_dir, label, name+f"-{color_idx}.jpg"), 
                        image[Y_min : Y_max, X_min : X_max])

100%|██████████| 3011/3011 [02:35<00:00, 19.39it/s]


empty masks:

/Users/samedi/Desktop/swans/maliy/images/2841811261.jpg     initial-maliy-301

/Users/samedi/Desktop/swans/maliy/images/3802649513.jpg     initial-maliy-646

/Users/samedi/Desktop/swans/shipun/images/img_2646.jpg      initial-shipun-2028

In [24]:
df_train = pd.read_csv(path_train_annotation)
test=[]
train=[]
for label in labels:
    df = df_train[df_train['class_name']==label].copy().reset_index()
    flag = df[df['path']==df['path'].unique()[100]].index[-1] + 1
    test.append(df.iloc[:flag])
    train.append(df.iloc[flag:])
df_train = pd.concat(train)
df_test = pd.concat(test)

In [30]:
df_test.drop(columns=['index']).to_csv(path_test_annotation, index=False)
df_train.drop(columns=['index']).to_csv(path_train_annotation, index=False)

In [9]:
#train->test
df_test = pd.read_csv(path_test_annotation)
for path in df_test['path'].unique():
    shutil.move(os.path.join(path_train_dataset,path.replace('.jpg','.txt')),
                os.path.join(path_test_dataset,path.replace('.jpg','.txt')))
    # for i, croppath in enumerate(df_test[df_test['path']==path]['path']):
    #     croppath = croppath.replace('.jpg', '-' + str(i) + '.jpg')
    #     shutil.move(os.path.join(path_cls_train_dataset,croppath),
    #                 os.path.join(path_cls_test_dataset,croppath))

In [11]:
# #test->train
# for path in df_test['path'].unique():
#     shutil.move(
#                 os.path.join(path_test_dataset,path),os.path.join(path_train_dataset,path))
#     for i, croppath in enumerate(df_test[df_test['path']==path]['path']):
#         croppath = croppath.replace('.jpg', '-' + str(i) + '.jpg')
#         shutil.move(
#                     os.path.join(path_cls_test_dataset,croppath),os.path.join(path_cls_train_dataset,croppath))

### remake segmentation annotation

In [27]:
# df_train = pd.read_csv(path_train_annotation)
number = {'path':[],'an':[],'sn':[]}
for path in tqdm(df_train['path'].unique()):
    with open(os.path.join(path_train_dataset, path.replace('.jpg','.txt'))) as f:
        an = df_train[df_train['path']==path]['path'].size
        lines = f.readlines()
        sn = 0
        for line in lines:
            sn += line!='\n'
        number['path'].append(path)
        number['an'].append(an)
        number['sn'].append(sn)
number = pd.DataFrame(number)
number.head()

100%|██████████| 7371/7371 [00:04<00:00, 1567.96it/s]


,path,an,sn
0,klikun/initial-klikun-122.jpg,2,3
1,klikun/initial-klikun-123.jpg,1,1
2,klikun/initial-klikun-125.jpg,1,1
3,klikun/initial-klikun-126.jpg,1,1
4,klikun/initial-klikun-127.jpg,1,1


In [28]:
(number['an']<number['sn']).sum() # after remaking

793

In [23]:
df_train=pd.read_csv(path_train_annotation)
df_val=pd.read_csv(path_val_annotation)
df_test=pd.read_csv(path_test_annotation)

dataset = path_initial_train_dataset
for label_idx, label in enumerate(labels):
    mask_dir = os.path.join(dataset, label, 'masks')
    image_dir = os.path.join(dataset, label, 'images')

    for file_idx, mask in enumerate(tqdm(os.listdir(mask_dir))):
        impath = os.path.join(image_dir, mask.replace('.png','.jpg'))
        if not os.path.exists(impath):
            # print(impath)
            continue # drop masks without images

        image = cv2.imread(impath)
        mask = cv2.imread(os.path.join(mask_dir, mask))

        h, w, _ = image.shape
        h_, w_, _ = mask.shape
        if not (h == h_ and w == w_):
            continue # drop images with uncorrect masks

        # name format: "[scource: initial|extra|augmented]-[label: klikun|maliy|shipun]-[ingroup index]]"
        name = f'initial-{label}-{file_idx}'

        save_dir=''
        if label+'/'+name+'.jpg' in df_train['path'].values:
            save_dir = path_train_dataset 
        elif label+'/'+name+'.jpg' in df_val['path'].values:
            save_dir = path_val_dataset
        elif label+'/'+name+'.jpg' in df_test['path'].values:
            save_dir = path_test_dataset
        else:
            # print(impath, name)
            continue


        path = os.path.join(save_dir, label, name)
        with open(path+'.txt','w') as labelfile:

            imgray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            for color_idx, color in enumerate(np.unique(imgray)[1:]):
                _, thresh = cv2.threshold(imgray, color, color, type=cv2.THRESH_TOZERO_INV)
                _, thresh = cv2.threshold(thresh, color-1, color, type=cv2.THRESH_TOZERO)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
                # RETR_EXTERNAL -> returns only most parental (external) contours
                
                for contour in contours:
                    seg = (contour[:,0]/[w,h]).flatten().tolist()
                    print(labeldict[label],*seg,file=labelfile) # save each spot for segmentation task

100%|██████████| 3011/3011 [01:56<00:00, 25.78it/s]
